In [ ]:
# import re
# import os
# import json
# def process_spam(body):
#     body_pp = body.lower()
#     body_pp = body_pp.replace("subject", "")
#     body_pp = body_pp.replace("nbsp", " ")
#     body_pp = re.sub(r"<[^<>]+>", " html ", body_pp)
#     body_pp = re.sub(r"[0-9]+", " number ", body_pp)
#     body_pp = re.sub(r"(http|https)://[^\s]*", ' httpaddr ', body_pp)
#     body_pp = re.sub(r"[^\s]+@[^\s]+", ' emailaddr ', body_pp)
#     body_pp = re.sub(r"[$]+", ' dollar ', body_pp)
#     body_pp = re.sub(r"[^a-zA-Z0-9]",' ', body_pp)
#     return body_pp

In [ ]:
# with open("lithosphere.spam1.csv", "w") as fpw:
#     fpw.write("messageId_@_threadId_@_parentId_@_subject_@_body_@_language_@_nodeStyle_@_userRank_@_tags_@_labels_@_spam_@_spamMarkUser_@_notMarkedAsSpam_@_uri\n")
#     with open("lithosphere.prod_spam_tag_labels.json") as fpr:
#         for line in fpr:
#             row = json.loads(line)
#             #print()
#             subject = process_spam(row['subject'])#normalize_document(row['subject'])
#             body = process_spam(row['body'])#normalize_document(row['body'])
#             lang = row["language"]
#             fpw.write("{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}_@_{}\n".format(row['messageId'],
#                                                              row['threadId'],
#                                                              row['parentId'],
#                                                              subject,
#                                                              body,
#                                                              lang,
#                                                              row['nodeStyle'].strip("\n"),
#                                                              row['userRank'],
#                                                              row['tags'],
#                                                              row['labels'],
#                                                              row['spam'],
#                                                              row['spamMarkUser'],
#                                                              row['notMarkedAsSpam'],
#                                                              row['uri']
#                                                             ))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import wordcloud

In [ ]:
with open("lithosphere.prod_spam_tag_labels.json") as fpr:
    print(fpr.readline())

In [ ]:
col_list = ["messageId","threadId","parentId","creationTime","subject","body","language","nodePath","nodeStyle","userId","userRank","tags","labels","spam","spamMarkUser","notMarkedAsSpam","uri"]
# df1 = pd.read_json('lithosphere.prod_spam_tag_labels.json',lines = True)
# df1.to_csv('lithosphere.spam2.csv') 

In [ ]:
original_data = pd.read_csv("lithosphere.spam2.csv", usecols=col_list)

In [ ]:
df=original_data.drop(columns=["nodePath","uri"])

In [ ]:
df1 = df.select_dtypes(include=[bool])
booleanDictionary = {True: 'TRUE', False: 'FALSE'}

for column in df1:
    df[column] = df[column].map(booleanDictionary)

In [ ]:
print(df.groupby('spam').describe())
print('----------------------------------------------------------')
print(df.info())

In [ ]:
df.head()

# Spam vs Ham

In [ ]:
print(df['spam'].value_counts())
sn.catplot(x="spam", kind="count",palette="pastel", edgecolor=".6",data=df,height=5,aspect=1)

# Automatic vs moderator

In [ ]:
print(df['notMarkedAsSpam'].value_counts())
sn.catplot(x="notMarkedAsSpam", kind="count",palette="muted", edgecolor=".6",data=df,height=5,aspect=1)

## Entries with the error

In [ ]:
dataModerated=df[df.notMarkedAsSpam != 'FALSE']
dataModerated

In [ ]:
print('-----------------------language-----------------------------------')
print(df['language'].value_counts())
print('--------------------------nodeStyle--------------------------------')
print(df['nodeStyle'].value_counts())
print('---------------------------userRank-------------------------------')
print(df['userRank'].value_counts().shape)
# print(df['userRank'].value_counts())
print('---------------------------tags-------------------------------')
print(df['tags'].value_counts().shape)
# print(df['tags'].value_counts())
print('----------------------------labels------------------------------')
print(df['labels'].value_counts().shape)
# print(df['labels'].value_counts())

In [ ]:
def tidy_split(df, column, sep=',', keep=False):
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        presplit=presplit.replace("[","").replace("]","")
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [ ]:
df_split_label=tidy_split(df,"labels",',',False)
df_split_tags=tidy_split(df,"tags",',',False)

## Check the spam count for NaN tags and labels

In [ ]:
null_tags=df[pd.isnull(df["tags"])]
null_tags["spam"].value_counts()

In [ ]:
null_label=df[pd.isnull(df["labels"])]
null_label["spam"].value_counts()

## Plot Spam count for top 30 tags

In [ ]:
plt.figure(figsize=(10,5))
df_spam=pd.DataFrame(df_split_tags.groupby('spam').get_group('TRUE'))
tempDf=df_spam['tags'].value_counts(sort=True)[:30]
chart = sn.barplot(tempDf.index, tempDf.values, alpha=0.8, color = "g")
plt.xticks(rotation=90)

## Plot Ham count for top 30 labels

In [ ]:
plt.figure(figsize=(10,5))
df_spam=pd.DataFrame(df_split_label.groupby('spam').get_group('FALSE'))
tempDf=df_spam['labels'].value_counts(sort=True)[:30]
chart = sn.barplot(tempDf.index, tempDf.values, alpha=0.8, color = "r")
plt.xticks(rotation=90)

In [ ]:
labels = ['Self-employed', 'Works at tech company', 'Has a tech role in non-tech company', 'Has a non-tech role at a non-tech company']
sizes = [data['Are you self-employed?'].value_counts()[1],
         data['Is your employer primarily a tech company/organization?'].value_counts()[1],
         data['Is your primary role within your company related to tech/IT?'].value_counts()[1],
         data['Is your primary role within your company related to tech/IT?'].value_counts()[0]
        ]
# print(sizes) # adds up to 1433, which is the total number of participants
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True)
ax1.axis('equal')
plt.show()

## Plot Spam per user count

In [ ]:
df_user_spam=pd.DataFrame(df.groupby('spam').get_group('TRUE'))
sn.catplot(x="userRank",kind="count",palette="pastel", edgecolor=".6", data=df_user_spam, height=5, aspect=1.5)
plt.xticks(rotation=90)

## Plot Spam per spamMarkUser

In [ ]:
df["spamMarkUser"].value_counts()

## Plot spam count for every nodeStyle

In [ ]:
df_style_spam=pd.DataFrame(df.groupby('spam').get_group('TRUE'))
sn.catplot(x="nodeStyle",kind="count",palette="pastel", edgecolor=".6", data=df_style_spam, height=5, aspect=1.5)
plt.xticks(rotation=90)

## Check spam if body is empty

In [ ]:
null_body=df[pd.isnull(df["body"])]
null_body["spam"].value_counts()
null_body.head()

In [ ]:
df['subject_length'] = df['subject'].apply(len)
df.head()

In [ ]:
plt.rcParams['patch.force_edgecolor'] = True
plt.style.use('seaborn-bright')
df.hist(column='subject_length', by='spam', bins=50,figsize=(11,5))

In [ ]:
df_nonNull_body = df.dropna(subset=['body'])
df_nonNull_body['body_length'] = df_nonNull_body['body'].apply(len)
df_nonNull_body.head()

In [ ]:
plt.rcParams['patch.force_edgecolor'] = True
plt.style.use('seaborn-bright')
df_nonNull_body.hist(column='body_length', by='spam', bins=50,figsize=(11,5))

## Wordcloud

In [ ]:
wordcloud.STOPWORDS

In [ ]:
def show_wordcloud(data_spam_or_ham, title):
    text = ' '.join(data_spam_or_ham['body'].astype(str).tolist())
    stopwords = set(wordcloud.STOPWORDS)
    
    fig_wordcloud = wordcloud.WordCloud(stopwords=stopwords,background_color='lightgrey',
                    colormap='viridis', width=800, height=600).generate(text)
    
    plt.figure(figsize=(10,7), frameon=True)
    plt.imshow(fig_wordcloud)  
    plt.axis('off')
    plt.title(title, fontsize=20 )
    plt.show()

In [ ]:
show_wordcloud(pd.DataFrame(df.groupby('spam').get_group('TRUE')), "Spam messages")

## Preprocessing

In [ ]:
from nltk.corpus import stopwords
import string
from collections import Counter

In [ ]:
def remove_punctuation_and_stopwords(sms):
    
    sms_no_punctuation = [ch for ch in sms if ch not in string.punctuation]
    sms_no_punctuation = "".join(sms_no_punctuation).split()
    
    sms_no_punctuation_no_stopwords = \
        [word.lower() for word in sms_no_punctuation if word.lower() not in stopwords.words("english")]
        
    return sms_no_punctuation_no_stopwords

In [ ]:
list_spam_words = []
for sublist in words_data_spam:
    for item in sublist:
        list_spam_words.append(item)

In [ ]:
c_spam = Counter(list_spam_words)
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x='word', y='count', 
            data=df_spamwords_top30, ax=ax)
plt.title("Top 30 Spam words")
plt.xticks(rotation='vertical');